In [5]:
import os
import numpy as np
import mne
import sklearn
import matplotlib.pyplot as plt
import pywt
from collections import Counter

from sklearn import svm
from sklearn.model_selection import train_test_split

from AudioOnsetUtils import *

RAW_EOG_CHANNELS = [u'EXG1', u'EXG2', u'EXG3', u'EXG4']
MASTOID_CHANNELS = [u'EXG5', u'EXG6']

In [6]:
## This cell gets filtered data so we can open ICA for Participant PNUM

def get_participant_filtered_data(PNUM):
    data_raw_file = os.path.join('raw_data', 'P' + PNUM + '-raw.fif')
    raw = mne.io.read_raw_fif(data_raw_file)

    # remove mastoid channels if present 
    if MASTOID_CHANNELS[0] in raw.ch_names:
        mne.io.set_eeg_reference(raw.load_data(), MASTOID_CHANNELS, copy=False) # inplace
        raw.drop_channels(MASTOID_CHANNELS)

    # Drop bad channels - in place on raw
    for bad_channel in raw.info['bads']:
        raw.drop_channels(bad_channel)
#         print("dropped: " + bad_channel)

    eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False, stim=False, exclude=[])

    # bandpass filter - keeping a frequency range between 0.5 (high pass filter) and 30 Hz (low pass filter)
    filtered_data = raw.load_data().filter(0.5, 30, picks=eeg_picks)
    
    return filtered_data

In [7]:
## This cell opens the ICA file for Participant PNUM

def get_ica_data_from_file(filtered_data, PNUM):
    ica = mne.preprocessing.read_ica('ica_data/P' + PNUM + '-ica.fif')

    # apply the transformation
    postica_data = ica.apply(filtered_data, exclude=ica.exclude)
    return postica_data

In [8]:
## Original code from https://github.com/sstober/openmiir-rl-2016/blob/master/openmiir/events.py#L21 and adapted 
KEYSTROKE_BASE_ID = 2000
TIME_INDEX = 0
ID_INDEX = 1

# returns two things, a True if Not Noise boolean and a tuple with three elements:
# 0- this is lyrical (0-2) <- YLABEL
# 1- start time (onsettime + cue duration if condition 1/2, else just onsettime)
# 2- end time (starttime + duration of song without cue)
def get_event_data(event_times_ids, events_index, music_version):
    event_id = event_times_ids[events_index[0]][ID_INDEX]

    if event_id < 1000:
        """
        Event Ids < 1000 are trial labels
        with the last digit indicating the condition
                1 : 'perception',
                2 : 'cued imag',
                3 : 'imag fix cross',
                4 : 'imagination',
        and the remaining digits referring to the stimulus id.
        """
        stimulus_id, condition = decode_event_id(event_id)
        
        stimulus_info = get_vers_stim_dict(music_version)[stimulus_id]
        cue_length = stimulus_info['cue_length']
        song_length = stimulus_info['song_length']
        
        events_index[0] += 1
        
        #print(get_id_to_song_name(stimulus_id))
        
        # get time of audio onset for this stimulus
        while event_times_ids[events_index[0]][ID_INDEX] != 1000:
            #print("Expected an audio onset event but got {}".format(event_times_ids[events_index[0]]))
            events_index[0] += 1
            
        audio_onset_time = event_times_ids[events_index[0]][TIME_INDEX]
        start_time = (audio_onset_time + cue_length) if condition in [1,2] else audio_onset_time
        end_time = start_time + song_length
        
        # move the pointer by 1 to prep next call to get_event_data
        events_index[0] += 1
        
        return True, (stimulus_info["audio_type"], start_time, end_time)
    else:
        # move the pointer by 1 to prep next call to get_event_data
        events_index[0] += 1
        
        return {
            1111: (False, ("noise")),
            KEYSTROKE_BASE_ID: (False, 'imagination failed'),
            KEYSTROKE_BASE_ID+1: (False, 'imagination okay')
        }[event_id]
    
    
# convert event id to stimulus (song num) and condition (1-4)
def decode_event_id(event_id):
    if event_id < 1000:
        stimulus_id = event_id // 10
        condition = event_id % 10
        return stimulus_id, condition
    else:
        return event_id

In [9]:
def get_event_data_list(event_times_ids, p):
    events_data_list = []
    events_index = [0]
    music_version = get_participant_vers(p)

    while len(events_data_list) < 50:
        is_audio, event_data = get_event_data(event_times_ids, events_index, music_version)
        if is_audio:
            # ignore AudioType.NONLYRICAL_LYRICAL (songs that have lyrics but played w/o lyrics)
            if event_data[0] == AudioType.LYRICAL or event_data[0] == AudioType.NON_LYRICAL:
                events_data_list.append(event_data)
    
    return events_data_list

In [13]:
def get_channels_eeg_data(event_matrix, start_time, end_time, ch_names, postica_data):
    # for each channel
    # extract this individual event's eeg data btwn start and endtime
    sampling_freq = postica_data.info['sfreq']

    for i in range(len(ch_names)):
        start_stop_seconds = np.array([11, 13])
        start_sample, stop_sample = (start_stop_seconds * sampling_freq).astype(int)
        
        # returns a tuple, see https://mne.tools/dev/auto_tutorials/raw/10_raw_overview.html
        #print(postica_data[i, start_sample:stop_sample])
        ch_selection = postica_data[i, start_sample:stop_sample]
        
        print("==================")
        print(ch_selection)
        
        # append eeg data list to eventMatrix
        event_matrix.append(ch_selection) 

In [14]:
## This cell prepares X_list and Y_list (data and labels)

X_list = []
Y_list = []
monster_matrix = [] # list of matrices

#for part in get_participant_list():
for part in ["01"]:
    # load ica data for participant
    filtered_data = get_participant_filtered_data(part)
    postica_data = get_ica_data_from_file(filtered_data, part)
    
    # from ica data, get stimuli events
    events = mne.find_events(postica_data)

    # creates tuples of (event_time, event_id)
    event_times_ids = [(events[:][i][0], events[:][i][2]) for i in range(len(events))]
    
    # get a list of event datas (enum, starttime, endtime)
    events_list = get_event_data_list(event_times_ids, part)
    
    # for each event in event datas
    for audio_type,start_time,end_time in events_list:
        event_matrix = [] # list of lists

        # get eeg data for all 62 channels, put into event_matrix
        eeg_ch_names = postica_data.ch_names[:62]
        get_channels_eeg_data(event_matrix, start_time, end_time, eeg_ch_names, postica_data)

        # call normalize on event_matrix
      
        # append event_matrix to monster_matrix
        monster_matrix.append(event_matrix)
        
        # 0 for non-lyrical, 1 for lyrical
        Y_list.append(1) if audio_type == AudioType.LYRICAL else Y_list.append(0)

Opening raw data file raw_data\P01-raw.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 64)  idle
    Range : 0 ... 2478165 =      0.000 ...  4840.166 secs
Ready.
Removing projector <Projection | Average EEG reference, active : False, n_channels : 64>
Reading 0 ... 2478165  =      0.000 ...  4840.166 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 3381 samples (6.604 sec)

Reading ica_data/P01-ica.fif ...
Isotrak not found
Now restoring ICA solut

(array([[ 1.24707494e-05,  1.15912833e-05,  1.07649899e-05, ...,
        -4.62164337e-06, -4.82593241e-06, -5.29947406e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-6.74483745e-06, -6.36237067e-06, -5.80842747e-06, ...,
        -7.99816127e-06, -7.06235880e-06, -6.24659215e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-3.45364788e-09,  5.29174068e-07,  1.23886251e-06, ...,
        -5.66973794e-06, -4.54826600e-06, -3.50695289e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.23913265e-06, -3.64543613e-06, -2.98700876e-06, ...,
        -6.45331857e-06, -5.41279826e-06, -4.47674702e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.46794844e-06, -3.76181940e-06, -3.00943202e-06, ...,
        -1.00578126e-05, -8.980

(array([[-5.50103749e-06, -5.68607173e-06, -5.70067890e-06, ...,
        -1.05095033e-06, -1.27283488e-06, -1.22318575e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[ 2.11991408e-06,  1.98202485e-06,  2.06058636e-06, ...,
         7.54751114e-07,  9.10992448e-08, -1.96822844e-07]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-0.01728034, -0.01729916, -0.01731669, ..., -0.01733237,
        -0.01732594, -0.01731666]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-1.71445517e-05, -1.61655615e-05, -1.50836781e-05, ...,
        -2.91177724e-06, -2.08584853e-06, -1.34200273e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-1.38189753e-05, -1.28202038e-05, -1.18493458e-05, ...,
         3.19050779e-06,  4.01699899e-06,  4.50294986e-

(array([[-4.23913265e-06, -3.64543613e-06, -2.98700876e-06, ...,
        -6.45331857e-06, -5.41279826e-06, -4.47674702e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.46794844e-06, -3.76181940e-06, -3.00943202e-06, ...,
        -1.00578126e-05, -8.98075049e-06, -8.01303477e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-3.50985863e-06, -3.10479063e-06, -2.67666040e-06, ...,
        -6.97279266e-06, -6.06731024e-06, -5.17220672e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.31793559e-06, -3.98325112e-06, -3.48420671e-06, ...,
        -3.15048101e-06, -2.53342744e-06, -1.94256044e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-1.14351416e-05, -1.10870220e-05, -1.05668224e-05, ...,
        -8.26836673e-06, -7.322

(array([[-1.48596495e-05, -1.43259866e-05, -1.38231978e-05, ...,
        -8.37085431e-07,  6.29356817e-07,  1.93384971e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-1.71848387e-06, -1.10432658e-06, -5.88268372e-07, ...,
        -1.01346185e-06, -1.45594583e-07,  4.73178542e-07]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[6.18912124e-07, 1.32355595e-06, 1.93213943e-06, ...,
        1.17159710e-06, 1.95298101e-06, 2.59092085e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.23640891e-06, -3.75738266e-06, -3.31594832e-06, ...,
        -4.98253873e-06, -4.08777203e-06, -3.37299533e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-7.20613951e-06, -6.45444043e-06, -5.72070167e-06, ...,
        -3.67841802e-06, -2.67758104e

...

       12.99609375, 12.99804688]))
(array([[ 5.87101756e-06,  5.67031869e-06,  5.34173690e-06, ...,
        -3.17586503e-06, -3.47568028e-06, -3.41899694e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-6.40474422e-06, -6.07003509e-06, -5.68992174e-06, ...,
         3.46149036e-06,  3.50534535e-06,  3.46258784e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[ 2.55928884e-06,  2.65544458e-06,  2.82085114e-06, ...,
        -1.04366056e-06, -7.55557654e-07, -4.93981500e-07]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.85555865e-06, -4.51211280e-06, -4.07148484e-06, ...,
        -6.66823036e-06, -6.16585893e-06, -5.75316527e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.19778743e-06, -3.27614414e-06, -2.36712679e-06, .

(array([[1.44436340e-06, 1.00315749e-06, 6.40621461e-07, ...,
        1.32591649e-06, 1.96672696e-06, 2.67661197e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-3.29616143e-06, -3.46501388e-06, -3.59656645e-06, ...,
         3.54584337e-06,  3.13499988e-06,  2.58817103e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[3.22455078e-06, 3.44286224e-06, 3.43299916e-06, ...,
        3.59759365e-07, 7.07156617e-07, 1.03465761e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[5.38944135e-06, 4.39382975e-06, 3.12813966e-06, ...,
        1.15619917e-05, 1.04502859e-05, 1.03996180e-05]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[ 5.87101756e-06,  5.67031869e-06,  5.34173690e-06, ...,
        -3.17586503e-06, -3.47568028e-06, -3.4189

(array([[-4.19778743e-06, -3.27614414e-06, -2.36712679e-06, ...,
        -3.85697211e-06, -3.25493541e-06, -2.85012119e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-3.82942263e-06, -3.56051372e-06, -3.31600055e-06, ...,
         1.71254001e-06,  2.10524820e-06,  2.23043637e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-1.45003739e-06, -1.46655600e-06, -1.49408854e-06, ...,
         2.50878352e-06,  3.48240256e-06,  4.41540146e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-8.15831782e-06, -7.30599939e-06, -6.42269404e-06, ...,
        -5.23103875e-06, -4.53912709e-06, -4.14524492e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-9.89609067e-06, -8.93310289e-06, -8.00374968e-06, ...,
        -2.24301920e-06, -1.328

(array([[ 5.87101756e-06,  5.67031869e-06,  5.34173690e-06, ...,
        -3.17586503e-06, -3.47568028e-06, -3.41899694e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-6.40474422e-06, -6.07003509e-06, -5.68992174e-06, ...,
         3.46149036e-06,  3.50534535e-06,  3.46258784e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[ 2.55928884e-06,  2.65544458e-06,  2.82085114e-06, ...,
        -1.04366056e-06, -7.55557654e-07, -4.93981500e-07]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.85555865e-06, -4.51211280e-06, -4.07148484e-06, ...,
        -6.66823036e-06, -6.16585893e-06, -5.75316527e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.19778743e-06, -3.27614414e-06, -2.36712679e-06, ...,
        -3.85697211e-06, -3.254

(array([[-3.89383999e-06, -3.56313199e-06, -3.02619523e-06, ...,
        -5.58361351e-06, -5.00873267e-06, -4.38625280e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-4.43555997e-06, -2.95799514e-06, -1.21586842e-06, ...,
         1.18258286e-06,  2.11923473e-06,  3.16023165e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-1.22768107e-05, -1.41908976e-05, -1.55972459e-05, ...,
        -1.14356431e-05, -1.02966638e-05, -8.82266899e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-1.64010937e-06, -2.08853061e-06, -1.88565591e-06, ...,
        -4.77052325e-06, -5.35722992e-06, -5.57041420e-06]]), array([11.        , 11.00195312, 11.00390625, ..., 12.99414062,
       12.99609375, 12.99804688]))
(array([[-5.50103749e-06, -5.68607173e-06, -5.70067890e-06, ...,
        -1.05095033e-06, -1.272

### Feature Extraction

In [ ]:
# code adapted from https://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/

def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy
 
def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]
 
def calculate_crossings(list_values):
    val_array = list_values
    num_zero_crossings = ((val_array[:-1] * val_array[1:]) < 0).sum()
    num_mean_crossings = ((val_array[:-1] * val_array[1:]) < np.nanmean(list_values)).sum()  
    return [num_zero_crossings, num_mean_crossings]
 
def get_features(list_values):
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [ ]:
# adapted from https://ataspinar.com/2018/12/21/a-guide-for-using-the-wavelet-transform-in-machine-learning/
WAVELET_NAME = 'db2' 

def get_dataset_features(dataset):
    feature_vectors = []
    
    for event_eeg_index in range(0, len(dataset)):
        event_features = []
        
        for channel_index in range(0, len(dataset[event_eeg_index])):
            channel_signal = dataset[event_eeg_index][channel_index]
            print(channel_signal)
            
            coeff_list = pywt.wavedec(channel_signal, WAVELET_NAME)
            #print(coeff_list)
            for coeff in coeff_list:
                #print(coeff)
                pass
                #event_features += get_features(coeff)
                
        feature_vectors.append(event_features)
        
    X = np.array(feature_vectors)
    return X

In [ ]:
print(monster_matrix)

In [ ]:
get_dataset_features(monster_matrix)

In [ ]:
# split data using (training vs testing)
# test_size: what proportion of original data is used for test set
train_data, test_data, train_lbl, test_lbl = train_test_split(X_list, Y_list, test_size=0.20, random_state=0)

### SVM and Check Accuracy

In [ ]:
def calcAccuracy(predictions, y):
    # True positives, false positives, etc.
    TP_ = np.logicaland(predictions, y)
    FP = np.logical_and(predictions, np.logicalnot(y))
    TN = np.logical_and(np.logical_not(predictions), np.logicalnot(y))
    FN = np.logical_and(np.logicalnot(predictions), y)

    TP = sum(TP)
    FP = sum(FP)
    TN = sum(TN)
    FN = sum(FN_)
    
    accuracy = (TP + TN)/(TP + FP + TN + FN)
    print('Accuracy:{}'.format(accuracy))
    
    return accuracy

In [ ]:
# perform classification using SVM
clf = svm.LinearSVC()
clf.fit(train_data, train_lbl)
y_pred = clf.predict(test_data)

# compare predictions for accuracy
accuracy = calcAccuracy(y_pred, Y_list)

# future work: we could compare accuracy linearSVC and just SVC